# Import and install packages

In [ ]:
!pip install spacy-transformers
!pip install spacy==3.0.5

     |████████████████████████████████| 71kB 3.2MB/s 
     |████████████████████████████████| 1.8MB 11.3MB/s 
     |████████████████████████████████| 460kB 28.2MB/s 
     |████████████████████████████████| 983kB 28.0MB/s 
     |████████████████████████████████| 890kB 41.7MB/s 
     |████████████████████████████████| 2.9MB 39.2MB/s 
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=3a9bda80867b75cd12eedba123545e17f81cac9ab654604bb60017c72b6ad103
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
  Created wheel for torchcontrib: filename=torchcontrib-0.0.2-cp37-none-any.whl size=7532 sha256=0d5d5e5aba45a03e3f78c1b79ac589485aa94ac8290c455277ab5e6e77feb751
  Stored in directory: /root/.cache/pip/wheels/06/06/7b/a5f5920bbf4f12a2c927e438fac17d4cd9560f8336b00e9a99
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a1af64f1c35b6529bcae939d22507668552026959b693bd05

In [ ]:
import pandas as pd
import numpy as np
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import re
import random
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U spacy[cuda101]
import spacy

Requirement already up-to-date: spacy[cuda101] in /usr/local/lib/python3.7/dist-packages (3.0.5)
     |████████████████████████████████| 121.6MB 88kB/s 


In [ ]:
gpu = spacy.prefer_gpu()
print('GPU:', gpu)


GPU: False


In [ ]:
#! python -m spacy init fill-config "/content/drive/My Drive/Notebooks/SCL 2021/base_config.cfg" "/content/drive/My Drive/Notebooks/SCL 2021/config.cfg"

# Read in and convert data

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Notebooks/SCL 2021/data/train.csv")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           300000 non-null  int64 
 1   raw_address  300000 non-null  object
 2   POI/street   300000 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.9+ MB


In [ ]:
data.head()

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru


In [ ]:
data.index.map(lambda x: "POI" if x%2==0 else "street")

Index(['POI', 'street', 'POI', 'street', 'POI', 'street', 'POI', 'street',
       'POI', 'street',
       ...
       'POI', 'street', 'POI', 'street', 'POI', 'street', 'POI', 'street',
       'POI', 'street'],
      dtype='object', length=300000)

In [ ]:
data["POI/street"] = data["POI/street"].str.split("/")
data = data.explode("POI/street").reset_index(drop=True)
data["label"] = data.index.map(lambda x: "POI" if x%2==0 else "street")
data

,id,raw_address,POI/street,label
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,POI
1,0,jl kapuk timur delta sili iii lippo cika 11 a ...,jl kapuk timur delta sili iii lippo cika,street
2,1,"aye, jati sampurna",,POI
3,1,"aye, jati sampurna",,street
4,2,setu siung 119 rt 5 1 13880 cipayung,,POI
...,...,...,...,...
599995,299997,tanjung gusta jl. yaya 2 no 17,,street
599996,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,POI
599997,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,,street
599998,299999,jaya maspion permata blok beryl b2 no.58,jaya maspion permata,POI


In [ ]:
data["POI/street"].value_counts()

                            248652
jend sudi                     1097
ahmad yani                     878
jend ahmad yani                618
gajah mada                     473
                             ...  
warung cantik                    1
pt.tri tunggal sinar mas         1
bisma 7                          1
depot ayam penyet 21             1
haji taiman                      1
Name: POI/street, Length: 187606, dtype: int64

In [ ]:
# drop rows with no entity
data = data[(data["POI/street"]).str.len()>0]
data

,id,raw_address,POI/street,label
1,0,jl kapuk timur delta sili iii lippo cika 11 a ...,jl kapuk timur delta sili iii lippo cika,street
5,2,setu siung 119 rt 5 1 13880 cipayung,siung,street
6,3,"toko dita, kertosono",toko dita,POI
9,4,jl. orde baru,jl. orde baru,street
10,5,"raya samb gede, 299 toko bb kids",toko bb kids,POI
...,...,...,...,...
599989,299994,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,kakap raya,street
599991,299995,jend ahmad yani 331 kertasari ciamis,jend ahmad yani,street
599993,299996,"raya cila kko, cilandak timur kel.",raya cila kko,street
599996,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,POI


In [ ]:
data.iloc[10]

id                                                         10
raw_address    cikahuripan sd neg boj 02 klap boj, no 5 16877
POI/street                                sd negeri bojong 02
label                                                     POI
Name: 20, dtype: object

In [ ]:
# fn to find index of entity
ids_missing = []
def get_index(row):
  try:
    start = re.search(row["POI/street"], row["raw_address"]).start()
    end = re.search(row["POI/street"], row["raw_address"]).end()
    return (start, end)
  except:
    ids_missing.append(row["id"])

In [ ]:
data["index"] = data.apply(lambda x: get_index(x), axis=1)
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,raw_address,POI/street,label,index
1,0,jl kapuk timur delta sili iii lippo cika 11 a ...,jl kapuk timur delta sili iii lippo cika,street,"(0, 40)"
5,2,setu siung 119 rt 5 1 13880 cipayung,siung,street,"(5, 10)"
6,3,"toko dita, kertosono",toko dita,POI,"(0, 9)"
9,4,jl. orde baru,jl. orde baru,street,"(0, 13)"
10,5,"raya samb gede, 299 toko bb kids",toko bb kids,POI,"(20, 32)"
...,...,...,...,...,...
599989,299994,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,kakap raya,street,"(14, 24)"
599991,299995,jend ahmad yani 331 kertasari ciamis,jend ahmad yani,street,"(0, 15)"
599993,299996,"raya cila kko, cilandak timur kel.",raya cila kko,street,"(0, 13)"
599996,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,POI,"(18, 28)"


In [ ]:
data_no_na = data[data["index"].notna()]
data_no_na

,id,raw_address,POI/street,label,index
1,0,jl kapuk timur delta sili iii lippo cika 11 a ...,jl kapuk timur delta sili iii lippo cika,street,"(0, 40)"
5,2,setu siung 119 rt 5 1 13880 cipayung,siung,street,"(5, 10)"
6,3,"toko dita, kertosono",toko dita,POI,"(0, 9)"
9,4,jl. orde baru,jl. orde baru,street,"(0, 13)"
10,5,"raya samb gede, 299 toko bb kids",toko bb kids,POI,"(20, 32)"
...,...,...,...,...,...
599989,299994,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,kakap raya,street,"(14, 24)"
599991,299995,jend ahmad yani 331 kertasari ciamis,jend ahmad yani,street,"(0, 15)"
599993,299996,"raya cila kko, cilandak timur kel.",raya cila kko,street,"(0, 13)"
599996,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,POI,"(18, 28)"


In [ ]:
data_no_na = data_no_na.groupby("raw_address", as_index=False).agg(list)
data_no_na

,raw_address,id,POI/street,label,index
0,""" bri kcp klampis kantor cabang pembantu klam...","[97899, 97899]","[kantor cabang pembantu klampis, jl. klampis j...","[POI, street]","[(19, 49), (51, 67)]"
1,""" aspol spn batua no. 16 makassar kelurahan te...",[126483],[aspol spn],[POI],"[(2, 11)]"
2,"""alamat: asrama tni ad kuta alam kecamatan: ku...",[21356],[asrama tni ad],[POI],"[(9, 22)]"
3,"""alamat: budi asih v no 9 kecamatan: sukasari ...",[143519],[budi asih],[POI],"[(9, 18)]"
4,"""alamat: jl.tegalsari rt.13 09 no.25, taman in...","[283805, 283805]","[taman intersos, jl.tegalsari]","[POI, street]","[(38, 52), (9, 21)]"
...,...,...,...,...,...
244834,"zurmin,",[256388],[zurmin],[POI],"[(0, 6)]"
244835,zuya busana salak pelabuhan baru,"[229703, 229703]","[zuya busana, salak]","[POI, street]","[(0, 11), (12, 17)]"
244836,"zyl, rajawali,",[93170],[rajawali],[street],"[(5, 13)]"
244837,"zzira collection, d. i. panja,",[36050],[zzira collection],[POI],"[(0, 16)]"


In [ ]:
len(data_no_na.iloc[0]["index"])

2

In [ ]:
# fn to check that labels don't overlap
def check_overlap(x):
  if len(x) == 1:
    return x
  if len(x) == 2:
    # check that both nos of X are not between nos of Y 
    if x[0][0] > x[1][0] and x[0][0] < x[1][1]:
      print(x)
      return np.nan
    elif x[1][0] > x[0][0] and x[1][0] < x[0][1]:
      print(x)
      return np.nan
    else:
      return x


In [ ]:
data_no_na["index_no_overlap"] = data_no_na["index"].apply(lambda x: check_overlap(x))
data_no_na = data_no_na.dropna(subset=["index_no_overlap"])
data_no_na

[(46, 60), (43, 60)]
[(0, 34), (5, 15)]
[(11, 26), (7, 21)]
[(0, 23), (14, 23)]
[(0, 21), (18, 21)]
[(0, 6), (3, 6)]
[(27, 39), (23, 47)]
[(29, 41), (25, 41)]
[(0, 24), (11, 20)]
[(0, 15), (7, 11)]
[(0, 27), (17, 23)]
[(42, 54), (38, 62)]
[(0, 23), (11, 15)]
[(23, 34), (20, 34)]
[(0, 27), (16, 24)]
[(0, 12), (7, 12)]
[(6, 26), (12, 22)]
[(0, 16), (12, 16)]
[(0, 16), (8, 16)]
[(0, 27), (16, 27)]
[(0, 15), (3, 12)]
[(0, 21), (4, 13)]
[(14, 35), (26, 33)]
[(0, 26), (4, 14)]
[(0, 17), (11, 17)]
[(0, 32), (6, 7)]
[(8, 38), (20, 31)]
[(21, 35), (17, 45)]
[(8, 22), (14, 18)]
[(8, 22), (14, 18)]
[(11, 30), (18, 21)]
[(15, 38), (29, 32)]
[(13, 28), (23, 28)]
[(8, 30), (26, 30)]
[(0, 23), (8, 17)]
[(0, 25), (8, 12)]
[(0, 43), (32, 43)]
[(0, 15), (3, 6)]
[(15, 59), (39, 48)]
[(0, 16), (8, 9)]
[(0, 32), (17, 27)]
[(0, 25), (20, 25)]
[(0, 50), (6, 9)]
[(8, 35), (16, 22)]
[(0, 27), (11, 21)]
[(12, 35), (25, 28)]
[(10, 30), (22, 26)]
[(7, 24), (20, 24)]
[(17, 33), (14, 38)]
[(0, 17), (7, 17)]
[(0, 11

,raw_address,id,POI/street,label,index,index_no_overlap
0,""" bri kcp klampis kantor cabang pembantu klam...","[97899, 97899]","[kantor cabang pembantu klampis, jl. klampis j...","[POI, street]","[(19, 49), (51, 67)]","[(19, 49), (51, 67)]"
1,""" aspol spn batua no. 16 makassar kelurahan te...",[126483],[aspol spn],[POI],"[(2, 11)]","[(2, 11)]"
2,"""alamat: asrama tni ad kuta alam kecamatan: ku...",[21356],[asrama tni ad],[POI],"[(9, 22)]","[(9, 22)]"
3,"""alamat: budi asih v no 9 kecamatan: sukasari ...",[143519],[budi asih],[POI],"[(9, 18)]","[(9, 18)]"
4,"""alamat: jl.tegalsari rt.13 09 no.25, taman in...","[283805, 283805]","[taman intersos, jl.tegalsari]","[POI, street]","[(38, 52), (9, 21)]","[(38, 52), (9, 21)]"
...,...,...,...,...,...,...
244834,"zurmin,",[256388],[zurmin],[POI],"[(0, 6)]","[(0, 6)]"
244835,zuya busana salak pelabuhan baru,"[229703, 229703]","[zuya busana, salak]","[POI, street]","[(0, 11), (12, 17)]","[(0, 11), (12, 17)]"
244836,"zyl, rajawali,",[93170],[rajawali],[street],"[(5, 13)]","[(5, 13)]"
244837,"zzira collection, d. i. panja,",[36050],[zzira collection],[POI],"[(0, 16)]","[(0, 16)]"


In [ ]:
# fn to combine into entity format
def combine(row):
  output = []
  for i in range(len(row["label"])):
    idx_1 = row["index"][i][0]
    idx_2 = row["index"][i][1]
    label = row["label"][i]

    output.append((idx_1, idx_2, label))
  return output

In [ ]:
data_no_na["entities_combined"] = data_no_na.apply(lambda x: combine(x), axis=1)
data_no_na

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,raw_address,id,POI/street,label,index,index_no_overlap,entities_combined
0,""" bri kcp klampis kantor cabang pembantu klam...","[97899, 97899]","[kantor cabang pembantu klampis, jl. klampis j...","[POI, street]","[(19, 49), (51, 67)]","[(19, 49), (51, 67)]","[(19, 49, POI), (51, 67, street)]"
1,""" aspol spn batua no. 16 makassar kelurahan te...",[126483],[aspol spn],[POI],"[(2, 11)]","[(2, 11)]","[(2, 11, POI)]"
2,"""alamat: asrama tni ad kuta alam kecamatan: ku...",[21356],[asrama tni ad],[POI],"[(9, 22)]","[(9, 22)]","[(9, 22, POI)]"
3,"""alamat: budi asih v no 9 kecamatan: sukasari ...",[143519],[budi asih],[POI],"[(9, 18)]","[(9, 18)]","[(9, 18, POI)]"
4,"""alamat: jl.tegalsari rt.13 09 no.25, taman in...","[283805, 283805]","[taman intersos, jl.tegalsari]","[POI, street]","[(38, 52), (9, 21)]","[(38, 52), (9, 21)]","[(38, 52, POI), (9, 21, street)]"
...,...,...,...,...,...,...,...
244834,"zurmin,",[256388],[zurmin],[POI],"[(0, 6)]","[(0, 6)]","[(0, 6, POI)]"
244835,zuya busana salak pelabuhan baru,"[229703, 229703]","[zuya busana, salak]","[POI, street]","[(0, 11), (12, 17)]","[(0, 11), (12, 17)]","[(0, 11, POI), (12, 17, street)]"
244836,"zyl, rajawali,",[93170],[rajawali],[street],"[(5, 13)]","[(5, 13)]","[(5, 13, street)]"
244837,"zzira collection, d. i. panja,",[36050],[zzira collection],[POI],"[(0, 16)]","[(0, 16)]","[(0, 16, POI)]"


In [ ]:
data_no_na["combined"] = data_no_na.apply(lambda x: (x["raw_address"], {"entities": x["entities_combined"]}), axis=1)
data_no_na

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,raw_address,id,POI/street,label,index,index_no_overlap,entities_combined,combined
0,""" bri kcp klampis kantor cabang pembantu klam...","[97899, 97899]","[kantor cabang pembantu klampis, jl. klampis j...","[POI, street]","[(19, 49), (51, 67)]","[(19, 49), (51, 67)]","[(19, 49, POI), (51, 67, street)]","("" bri kcp klampis kantor cabang pembantu kla..."
1,""" aspol spn batua no. 16 makassar kelurahan te...",[126483],[aspol spn],[POI],"[(2, 11)]","[(2, 11)]","[(2, 11, POI)]","("" aspol spn batua no. 16 makassar kelurahan t..."
2,"""alamat: asrama tni ad kuta alam kecamatan: ku...",[21356],[asrama tni ad],[POI],"[(9, 22)]","[(9, 22)]","[(9, 22, POI)]","(""alamat: asrama tni ad kuta alam kecamatan: k..."
3,"""alamat: budi asih v no 9 kecamatan: sukasari ...",[143519],[budi asih],[POI],"[(9, 18)]","[(9, 18)]","[(9, 18, POI)]","(""alamat: budi asih v no 9 kecamatan: sukasari..."
4,"""alamat: jl.tegalsari rt.13 09 no.25, taman in...","[283805, 283805]","[taman intersos, jl.tegalsari]","[POI, street]","[(38, 52), (9, 21)]","[(38, 52), (9, 21)]","[(38, 52, POI), (9, 21, street)]","(""alamat: jl.tegalsari rt.13 09 no.25, taman i..."
...,...,...,...,...,...,...,...,...
244834,"zurmin,",[256388],[zurmin],[POI],"[(0, 6)]","[(0, 6)]","[(0, 6, POI)]","(zurmin,, {'entities': [(0, 6, 'POI')]})"
244835,zuya busana salak pelabuhan baru,"[229703, 229703]","[zuya busana, salak]","[POI, street]","[(0, 11), (12, 17)]","[(0, 11), (12, 17)]","[(0, 11, POI), (12, 17, street)]","(zuya busana salak pelabuhan baru, {'entities'..."
244836,"zyl, rajawali,",[93170],[rajawali],[street],"[(5, 13)]","[(5, 13)]","[(5, 13, street)]","(zyl, rajawali,, {'entities': [(5, 13, 'street..."
244837,"zzira collection, d. i. panja,",[36050],[zzira collection],[POI],"[(0, 16)]","[(0, 16)]","[(0, 16, POI)]","(zzira collection, d. i. panja,, {'entities': ..."


In [ ]:
train = data_no_na["combined"].tolist()

# Train model

In [ ]:
# import spacy
# gpu = spacy.require_gpu()
# print('GPU:', gpu)

nlp = spacy.blank("id")
nlp.add_pipe("ner")

In [ ]:
ner = nlp.get_pipe("ner")
for _, annotations in train:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [ ]:
# train_split = 0.1
n_iter = 5

examples = []
for text, annots in tqdm(random.sample(train, 5000)):
  try:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))
    nlp.initialize(lambda: examples)
    for i in (range(n_iter)):
      random.shuffle(examples)
      for batch in minibatch(examples, size=compounding(4.0, 16.0, 1.001)):
          nlp.update(batch)
          # nlp.update(
          #             text,  # batch of texts
          #             annots,  # batch of annotations
          #             drop=0.5,  # dropout - make it harder to memorise data
          #             sgd=optimiser,  # callable to update weights
          #             losses=losses,
          #         )
      # print("Losses:", losses)
  except Exception as e:
    print(text, f"error: {e}")


  4%|▎         | 177/5000 [09:21<8:01:11,  5.99s/it]/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "jagakarsa s, 56 rt 6 7" with entities "[(7, 8, 'street')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,

  4%|▍         | 201/5000 [11:57<9:05:54,  6.83s/it]/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "jalan panglima denai masuk ke jalan jermal vii .je..." with entities "[(48, 64, 'POI'), (30, 46, 'street')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str

kerta foto copy, ker, ciputat timur error: [E103] Trying to set conflicting doc.ents: '(0, 15, 'POI')' and '(0, 3, 'street')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.



 20%|██        | 1007/5000 [4:46:13<37:07:22, 33.47s/it]/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "lr mustika sebelah warung ayam pak bos(hotel bina ..." with entities "[(39, 55, 'POI')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,

 22%|██▏       | 1118/5000 [5:51:30<39:15:41, 36.41s/it]/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "cluster tria adara residence 9,jalan raya serua(ma..." with entities "[(0, 30, 'POI'), (55, 68, 'street')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + ".

In [ ]:
data_no_na[data_no_na["raw_address"] =="toko ayam bik murni kuripan babas pagar alam utara"]

,raw_address,id,POI/street,label,index,index_no_overlap,entities_combined,combined
226636,toko ayam bik murni kuripan babas pagar alam u...,[172907],[toko ayam bik murni],[POI],"[(0, 19)]","[(0, 19)]","[(0, 19, POI)]",(toko ayam bik murni kuripan babas pagar alam ...


In [ ]:
# save model
nlp.to_disk("/content/drive/My Drive/Notebooks/SCL 2021/models")


In [ ]:
train[11]

('"blok a b1 a 100 tanah abang \t"', {'entities': [(1, 7, 'POI')]})

In [ ]:
train[-189621:-189619]

[('gand 1, no 341 rujak michael gancit',
  {'entities': [(15, 35, 'POI'), (0, 6, 'street')]}),
 ('gand 1, no 3e apmi, 1', {'entities': [(14, 18, 'POI'), (0, 6, 'street')]})]

# Evaluate model

In [ ]:
nlp_updated = spacy.load("/content/drive/My Drive/Notebooks/SCL 2021/1k_model")

In [ ]:
test = pd.read_csv("/content/drive/My Drive/Notebooks/SCL 2021/data/test.csv")

In [ ]:
test.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"


In [ ]:
test_address = test["raw_address"].tolist()

In [ ]:
# from collections import defaultdict
# # Number of alternate analyses to consider. More is slower, and not necessarily better -- you need to experiment on your problem.
# beam_width = 16
# # This clips solutions at each step. We multiply the score of the top-ranked action by this value, and use the result as a threshold. This prevents the parser from exploring options that look very unlikely, saving a bit of efficiency. Accuracy may also improve, because we've trained on greedy objective.
# beam_density = 0.0001 

# for text in data_no_na["raw_address"][:20]:
#   docs = list(nlp_updated.pipe(text, disable=['ner']))
#   beams = nlp_updated.get_pipe("ner").beam_parse(docs, beam_width=beam_width, beam_density=beam_density)
#   # doc2 = nlp_updated(text)
#   # for ent in doc2.ents:
#   #   t = ent.text

#   for doc, beam in zip(docs, beams):
#       entity_scores = defaultdict(float)
#       for score, ents in nlp_updated.get_pipe("ner").moves.get_beam_parses(beam):
#           for start, end, label in ents:
#               entity_scores[(start, end, label)] += score

#   l= []
#   for k, v in entity_scores.items():
#       l.append({'start': k[0], 'end': k[1], 'label': k[2], 'prob' : v} )
#       # l.append({'text': t, 'label': k[2], 'prob' : v} )

#   print (l)

In [ ]:
# fn to predict labels
def predict_labels(row):
  doc = nlp_updated(row["raw_address"])
  POI = ''
  street = ''
  for ent in doc.ents:
    text = ent.text
    label = ent.label_
    if label == "POI":
      POI = text
    elif label == "street":
      street = text

  return POI + "/" + street

In [ ]:
test.iloc[:10].apply(lambda x: predict_labels(x), axis=1)

0                   /s. par
1                 /angg per
2       asma laun/mand imog
3             /raya nga sri
4                /cut mutia
5                /perum gar
6                 /tb. mara
7    pura taman beji tista/
8                      /pam
9                 /raya won
dtype: object

In [ ]:
test["POI/street"] = test.apply(lambda x: predict_labels(x), axis=1)

In [ ]:
test

,id,raw_address,POI/street
0,0,s. par 53 sidanegara 4 cilacap tengah,/s. par
1,1,"angg per, baloi indah kel. lubuk baja",/angg per
2,2,"asma laun, mand imog,",asma laun/mand imog
3,3,"ud agung rej, raya nga sri wedari karanganyar",/raya nga sri
4,4,"cut mutia, 35 baiturrahman",/cut mutia
...,...,...,...
49995,49995,toko mbak farid semboro semboro,toko mbak farid semboro semboro/
49996,49996,"vie - tk. ridho kids, vete 3 cari, 16720 ciawi",/vete 3 cari
49997,49997,"mart dan roti bakar malabar, nasio,",mart dan roti bakar malabar/
49998,49998,graha indah pamulang jl. mujair raya bambu apu...,graha indah/jl. mujair


In [ ]:
submission = test[["id", "POI/street"]]
submission.to_csv("/content/drive/My Drive/Notebooks/SCL 2021/submission.csv", index=False)

In [ ]:
submission

,id,POI/street
0,0,/s. par
1,1,/angg per
2,2,asma laun/mand imog
3,3,/raya nga sri
4,4,/cut mutia
...,...,...
49995,49995,toko mbak farid semboro semboro/
49996,49996,/vete 3 cari
49997,49997,mart dan roti bakar malabar/
49998,49998,graha indah/jl. mujair


In [ ]:
for text in test["raw_address"][:10]:
  doc2 = nlp_updated(text)
  print("Entities", [(ent.text, ent.label_) for ent in doc2.ents])

Entities [('s. par', 'street')]
Entities [('angg per', 'street')]
Entities [('asma laun', 'POI'), ('mand imog', 'street')]
Entities [('raya nga sri', 'street')]
Entities [('cut mutia', 'street')]
Entities [('pem dos', 'street'), ('perum gar', 'street')]
Entities [('tb. mara', 'street')]
Entities [('pura taman beji tista', 'POI')]
Entities [('pam', 'street')]
Entities [('raya won', 'street')]
